In [ ]:
import os
from aicsimageio import AICSImage
from tifffile import imwrite
from cellpose import models
import numpy as np
from skimage.io import imread, imsave
from skimage.measure import regionprops
import json
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage.color import label2rgb
from skimage.io import imsave
import matplotlib.pyplot as plt
import pandas as pd
from skimage.segmentation import find_boundaries
from skimage.util import img_as_ubyte
from skimage.io import imsave
from skimage import img_as_float
from skimage import exposure

if torch.cuda.is_available() is True:
    try:
        import cupy as cu
        from cucim.skimage.morphology import dilation, disk
        cuda = True
    except ImportError:
        from skimage.morphology import dilation, disk
        cuda = False
else:
    from skimage.morphology import dilation, disk
    cuda = False

In [ ]:
## ENTER YOUR VALUES ##

parent_directory = "/path/to/your/folder/" # Replace with the actual path
cellpose_directory = "/path/to/your/model" # Replace with the actual path

myotube_channel = 0 # Replace with the actual myotube channel 
nuclei_channel = 1 # Replace with the actual nuclei channel 

dia = 24 # Adjust this value depending on your images. Set it to 0 for Cellpose to automatically determine the best value.

extension = (".tif", ".tiff")

In [ ]:
## IMAGE SPLITTING ## 

# If necessary - Split Image into separate channels for processing

# Define paths
full_images_folder = os.path.join(parent_directory, "Full Images")
myotube_folder = os.path.join(parent_directory, "Images")
nuclei_folder = os.path.join(parent_directory, "Nuclei")
predictions_output_dir = os.path.join(parent_directory, "Predictions")

# Ensure output folders exists
os.makedirs(myotube_folder, exist_ok=True)
os.makedirs(nuclei_folder, exist_ok=True)
os.makedirs(predictions_output_dir, exist_ok=True)

# Process each image in "Full Images"
if os.path.exists(full_images_folder):
    for file in os.listdir(full_images_folder):
        file_path = os.path.join(full_images_folder, file)

        # Check if it's an image file
        if file.lower().endswith((".tiff", ".tif")):
            image = imread(file_path)

        elif file.lower().endswith((".ome.tiff", ".czi")):
            aics_image = AICSImage(file_path)
            image = aics_image.data[0]  # Extract the data for the first scene
            if image.ndim == 5:  # TCZYX
                image = image[0, 0].transpose((1, 2, 0))
            elif image.ndim == 4:  # CZYX
                image = image[0].transpose((1, 2, 0))
        else:
            print(f"Skipped {file}: unsupported file format.")
            continue

        # Check if the image has at least 2 channels
        if image.ndim == 3 and image.shape[2] >= 2:
            myotube = image[myotube_channel, :, :]
            nuclei = image[nuclei_channel, :, :]

            # Save Myotube channel
            myotube_save_path = os.path.join(myotube_folder, f"{os.path.splitext(file)[0]}.tif")
            imsave(myotube_save_path, myotube.astype(np.uint16))

            # Save Nuclei channel
            nuclei_save_path = os.path.join(nuclei_folder, f"{os.path.splitext(file)[0]}.tif")
            imsave(nuclei_save_path, nuclei.astype(np.uint16))

            print(f"Processed and saved channels for: {file}")
        else:
            print(f"Skipped {file}: not enough channels.")
else:
    print(f"The folder 'Full Images' does not exist in {parent_directory}")

In [ ]:
## SEGMENTATION ##

# List to store all image file paths
image_files = []

# Check if the "Nuclei" folder exists
if os.path.exists(nuclei_folder):
    for file in os.listdir(nuclei_folder):
        file_path = os.path.join(nuclei_folder, file)
        # Check if the file is an image (by extension)
        if file.lower().endswith(( ".tiff", ".tif")):
            image_files.append(file_path)
else:
    print(f"The folder 'Nuclei' does not exist in {parent_directory}")

# Print the list of images
for index, file in enumerate(image_files):
    print(f"{file}")

In [ ]:
def load_image(file_path):
    
    # Load a single channel image
    image = AICSImage(file_path).data[0][0] 
    return image

def loop(file_path, parent_directory, diameter):
    base_name = os.path.basename(file_path)
    name_without_ext = os.path.splitext(base_name)[0]

    # Ensure the "Masks" folder exists
    masks_folder = os.path.join(parent_directory, "Masks")
    if not os.path.exists(masks_folder):
        os.makedirs(masks_folder)

    # Empty GPU cache
    torch.cuda.empty_cache()

    # Load the image
    img = load_image(file_path)

    # Cellpose
    model = models.CellposeModel(
        gpu=True,
        pretrained_model= cellpose_directory)
    masks, flows, styles = model.eval(img, diameter, channels=[0, 0], normalize=True)

     # Save the mask as compressed .tiff
    if masks.max() > 0:
        mask_save_path = os.path.join(masks_folder, name_without_ext + ".tif")
        imwrite(mask_save_path, masks.astype(np.uint16), compression='zlib')
        print(f"Mask saved: {mask_save_path}")
    else:
        print(f"No cells detected in {file_path}, no mask has been saved.")

    result = {
        'image': file_path,
        'diameter': diameter,
        'cells count': np.max(masks)
    }

    # Empty GPU cache
    torch.cuda.empty_cache()

    return result

In [ ]:
# List all image files in the "Nuclei" folder
image_files = [f for f in os.listdir(nuclei_folder) if f.lower().endswith(('.tif', '.tiff'))]

# Process each image
for file in image_files:
    file_path = os.path.join(nuclei_folder, file)
    print(f"Processing file: {file_path}")
    result = loop(file_path, parent_directory, dia)                                                                                                                     

In [ ]:
## CLASSIFICATION ##

def load_images_labels_props(parent_directory):
    images_dir = os.path.join(parent_directory, "Images")
    masks_dir = os.path.join(parent_directory, "Masks")

    image_files = [f for f in os.listdir(images_dir) 
                   if f.lower().endswith(('.tif', '.tiff'))]

    images = []
    labels = []
    all_props = []

    for img_file in image_files:
        img_path = os.path.join(images_dir, img_file)
        mask_path = os.path.join(masks_dir, img_file)

        # Check if there is a corresponding mask
        if not os.path.exists(mask_path):
            print(f"No corresponding mask for {img_file}")
            continue

        # Load Image and Mask
        image_data = imread(img_path)
        mask_data = imread(mask_path)

        # Convert image to RGB
        if len(image_data.shape) == 2:  # Check if grayscale
            image_data = np.stack((image_data,) * 3, axis=-1)  # Convert to RGB

        images.append(image_data)
        labels.append(mask_data)

        # Calculate regionprops properties
        mask_label = mask_data.astype(np.int32, copy=False)
        props = regionprops(mask_label)

        all_props.append(props)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels, all_props

In [ ]:
def min_max_norm(im):
    im = (im - im.min()) / (im.max() - im.min())
    return im

class PredictionDataset(Dataset):
    def __init__(self, image, labels, props, half_patch_size, device, config_dict):
        if not isinstance(props, (list, tuple)):
            raise ValueError("props must be a list or tuple, obtained from regionprops.")
        
        self.props = props
        self.image = image
        self.labels = labels
        self.half_patch_size = half_patch_size
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.device = device
        self.config_dict = config_dict
        self.used_labels = [prop.label for prop in props]

        pad_width = half_patch_size + 1
        self.image = np.pad(image, ((pad_width, pad_width), (pad_width, pad_width), (0, 0)), mode="constant")
        self.labels = np.pad(labels, ((pad_width, pad_width), (pad_width, pad_width)), mode="constant")

    def __getitem__(self, index):
        prop = self.props[index]
        try:
            cx, cy = map(int, prop.centroid)
            cx += self.half_patch_size + 1
            cy += self.half_patch_size + 1

            xmin, xmax = cx - self.half_patch_size, cx + self.half_patch_size
            ymin, ymax = cy - self.half_patch_size, cy + self.half_patch_size

            imagette = self.image[xmin:xmax, ymin:ymax].copy()
            maskette = self.labels[xmin:xmax, ymin:ymax].copy()
            maskette[maskette != prop.label] = 0
            maskette[maskette == prop.label] = 1

            if json.loads(self.config_dict["options"]["dilation"]["dilate_mask"].lower()):
                str_el = disk(int(self.config_dict["options"]["dilation"]["str_element_size"]))
                maskette = dilation(maskette, str_el)
                imagette *= maskette[:, :, None]

            concat_image = np.zeros((imagette.shape[0], imagette.shape[1], imagette.shape[2] + 1))
            concat_image[:, :, :-1] = imagette / imagette.max()
            concat_image[:, :, -1] = maskette

            return self.transform(concat_image.astype("float32")).to(self.device)

        except Exception as e:
            print(f"Error processing object {index}: {e}")
            return None

    def __len__(self):
        return len(self.props)

In [ ]:
def save_colored_predictions(labels, predictions, used_labels, myotube_image_path, output_dir, image_name):
    
    # Load the myotube grayscale image
    myotube_image = imread(myotube_image_path)
    
    # Adjust the contrast of the myotube image for better visibility
    myotube_image = exposure.rescale_intensity(myotube_image, in_range='image', out_range=(0, 1))

    # Convert the grayscale image to RGB (3 channels) for colorizing
    myotube_rgb = np.stack((myotube_image,) * 3, axis=-1)

    # Extract region properties from the label image
    props = regionprops(labels)

    # Extract the labels corresponding to the used regions
    extracted_labels = [prop.label for prop in props if prop.label in used_labels]

    # Ensure that the number of predictions matches the number of used labels
    if len(predictions) != len(extracted_labels):
        raise ValueError(f"Mismatch: {len(predictions)} predictions for {len(extracted_labels)} labels")

    # Find the boundaries of each labeled region
    boundaries = find_boundaries(labels, mode='inner')

    # Define colors for each prediction class
    color_0 = [1, 0, 0]  # Red for class 0
    color_1 = [0, 1, 0]  # Green for class 1

    # Apply colors to the boundaries of each region based on its prediction
    for region_label, prediction in zip(extracted_labels, predictions):
        # Create a mask for the current label's boundary
        mask = (labels == region_label) & boundaries
        if prediction == 0:
            myotube_rgb[mask] = color_0  # Red for class 0
        elif prediction == 1:
            myotube_rgb[mask] = color_1  # Green for class 1

    # Convert the RGB image to 8-bit format for saving
    myotube_rgb = img_as_ubyte(myotube_rgb)

    # Save the colorized image to the specified output directory
    output_path = os.path.join(output_dir, f"{image_name}_prediction.tif")
    imsave(output_path, myotube_rgb)

In [ ]:
model_path = os.path.join(parent_directory, 'Svetlana', 'MyoFuse.pth')                               

In [ ]:
if __name__ == "__main__":
    images, labels, all_props = load_images_labels_props(parent_directory)
    config_path = os.path.join(parent_directory, "Svetlana", "Config.json")
    with open(config_path, 'r') as f:
        config_dict = json.load(f)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    half_patch_size = 100

    checkpoint = torch.load(model_path, map_location=device)
    model = checkpoint["model"]
    model.to(device)
    model.eval()

    all_predictions = []
    all_used_labels = []

    for i in range(len(images)):
        print(f"Processing image {i + 1}/{len(images)}")
        current_image = images[i]
        current_label = labels[i]
        current_props = all_props[i]

        dataset = PredictionDataset(
            image=current_image,
            labels=current_label,
            props=current_props,
            half_patch_size=half_patch_size,
            device=device,
            config_dict=config_dict,
        )
        dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

        used_labels = dataset.used_labels
        image_predictions = []

        with torch.no_grad():
            for batch in dataloader:
                if batch is None or batch.size(0) == 0:
                    continue
                batch = batch.to(device)
                output = model(batch)
                pred = output.argmax(dim=1)
                image_predictions.extend(pred.cpu().numpy())

        image_name = os.path.splitext(os.path.basename(image_files[i]))[0]
        myotube_image_path = os.path.join(myotube_folder, f"{image_name}.tif")
        save_colored_predictions(
            labels=current_label,
            predictions=image_predictions,
            used_labels=used_labels,
            myotube_image_path=myotube_image_path,
            output_dir=predictions_output_dir,
            image_name=image_name
        )

        all_predictions.append(image_predictions)
        all_used_labels.append(used_labels)

In [ ]:
## SAVE RESULTS ##

# List to store statistics for each image
prediction_data = []

# Iterate over predictions for each image
for i, predictions in enumerate(all_predictions):
    num_ones = 0
    num_zeros = 0

    # Iterate over predictions in the image
    for pred in predictions:
        # Ensure that each `pred` is a valid array
        if pred.size > 0:
            num_ones += np.sum(pred == 1)  # Count pixels predicted as 1
            num_zeros += np.sum(pred == 0)  # Count pixels predicted as 0

    # Calculate the total number of labels
    total_labels = num_ones + num_zeros
    fusion_index = num_ones / total_labels * 100

    # Add statistics to the list
    prediction_data.append({
        "Image Name": os.path.splitext(os.path.basename(image_files[i]))[0],
        "Total Number of Nuclei": total_labels,
        "Nuclei In": num_ones,
        "Nuclei Out": num_zeros,
        "Fusion Index (%)": fusion_index,
    })

# Create a DataFrame from the data
predictions_df = pd.DataFrame(prediction_data)

# Export the DataFrame to an Excel file
output_file_path = os.path.join(parent_directory, "predictions_fusion_index.xlsx")
predictions_df.to_excel(output_file_path, index=False)

# Display the DataFrame without index
print(f"\nDataFrame exported to Excel file: {output_file_path}")